In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy import stats
ibge_path = "censo_uf/"

In [2]:
#carregar dados do ibge
ibge_names = ["municipio", "total", "semfund", "fund", "med", "sup", "nd", "codigo"]
df = []
for i in range(1, 28):
    filepath = ibge_path + ("Tabela 2.%s.5.4.xls" % (str(i)))
    raw = pd.read_excel(filepath, names=ibge_names)[6:-2]
    df.append(raw)
df = pd.concat(df)

In [3]:
#carregar votacao
vota_path = "eleicao/votacao_secao_2014_BR.txt"
vota_names = ["data", "hora", "ano", "turno", "desc",
              "uf", "ue", "cod_mun", "municipio", "zona",
              "secao", "cod_cargo", "cargo", "partido", "votos"]
raw = pd.read_csv(vota_path, delimiter=";", names=vota_names, encoding="latin-1")

In [ ]:
segundo_turno = raw[raw.turno == 2]

In [ ]:
N = len(df.municipio)
resul = np.zeros((N, 4))

i = 0
for cid in df.iterrows():
    frac = (cid[1].sup / cid[1].total)
    cidade = cid[1].municipio
    city_dat = segundo_turno[segundo_turno.municipio == cidade.upper()]
    dilma_v = np.sum(city_dat[city_dat.partido == 13].votos)
    aecio_v = np.sum(city_dat[city_dat.partido == 45].votos)
    total_v = np.sum(city_dat.votos)
    resul[i] = [frac, dilma_v, aecio_v, total_v]
    i += 1
    if(i % np.floor(N/10) == 0):
        print(N - i)

5009


In [ ]:
resul_df = pd.DataFrame(resul, columns=["frac_superior", "dilma", "aecio", "total"])

In [ ]:
resul_df = resul_df[(resul_df.dilma > 0) & (resul_df.aecio > 0)]
frac_voto = resul_df.aecio / (resul_df.aecio + resul_df.dilma)
(x, y) = (frac_voto, resul_df.frac_superior)

In [ ]:
#plt.scatter(y=y, x=x, marker='.')
plt.hist2d(x,y,bins=50)
plt.colorbar(label="Quantidade de municipios")
plt.xlim(0, 1)
plt.ylim(0, 0.3)
plt.title("Instrução superior nos municípios vs opção de voto em 2014")
plt.ylabel("Fração da população com ensino superior")
plt.xlabel("Fração dos que votaram no Aécio em relação a Aécio+Dilma")
plt.show()

In [242]:
fig, ax = plt.subplots()
fit = np.polyfit(x, y, deg=1)
ax.plot(x, fit[0] * x + fit[1], color='red')
ax.scatter(x, y, marker='.')
plt.show()

In [231]:
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)

In [239]:
slope * 0.5 + intercept

0.048325148724178722

In [240]:
np.mean(y)

0.041784330855349723